# Lesson I4: Closures & Iterators

**Duration**: 105-120 minutes  
**Stage**: Intermediate (Building Skills)  
**Prerequisites**: Lessons I1-I3 (Structs, Traits, Modules)

---

## 📋 What You'll Learn

This lesson introduces functional programming in Rust through closures and iterators. You'll learn to write expressive, efficient code using iterator chains and understand how Rust achieves zero-cost abstractions.

**Why this matters**: Iterators are one of Rust's most powerful features - they're as fast as hand-written loops but more expressive and less error-prone. Closures enable functional programming patterns used throughout the Rust ecosystem, from async code to data processing pipelines.

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
1. Work with closures and understand capture modes
2. Use iterator adapters and consumers effectively
3. Apply functional programming patterns
4. Create custom iterators for your types
5. Apply iterator chains for data processing
6. Understand performance characteristics of iterators

---

## 🧠 Key Concepts

### Iterator Fundamentals

**Iterator Trait**:
- **Lazy**: Iterators do nothing until consumed
- **Zero-Cost**: Compile to same code as manual loops
- **Composable**: Chain operations together
- **Functional**: Immutable transformations

### Iterator Categories

- **Adapters**: Transform iterators (`map`, `filter`, `enumerate`)
- **Consumers**: Consume iterators (`collect`, `fold`, `for_each`)
- **Creators**: Create iterators (`iter`, `into_iter`, `iter_mut`)

---

## 🎭 Closures: Anonymous Functions

**Closures** are anonymous functions that can capture their environment. They're essential for working with iterators.

### Closure Syntax

```rust
let add_one = |x| x + 1;              // Type inferred
let add = |x: i32, y: i32| x + y;     // Explicit types
let complex = |x| {                   // Multi-line body
    let result = x * 2;
    result + 1
};
```

### Closure Traits: Fn, FnMut, FnOnce

Closures implement one or more of these traits based on how they capture variables:

| Trait | Captures | Can be called | Example |
|-------|----------|---------------|----------|
| `Fn` | By immutable reference | Multiple times | `\|x\| x + y` (y borrowed) |
| `FnMut` | By mutable reference | Multiple times | `\|x\| { count += 1; x }` |
| `FnOnce` | By value (move) | Once | `\|x\| vec.push(x)` (vec moved) |

**📚 Rust Book Reference:** [Chapter 13.1 - Closures](https://doc.rust-lang.org/book/ch13-01-closures.html)

In [ ]:
// Closures: Capturing environment and different traits

fn closures_demo() {
    println!("=== Closures Demo ===");
    
    // Basic closure
    let add_one = |x| x + 1;
    println!("add_one(5) = {}", add_one(5));
    
    // Closure capturing environment (Fn trait)
    let multiplier = 10;
    let multiply = |x| x * multiplier;  // Captures multiplier by reference
    println!("multiply(5) = {}", multiply(5));
    println!("Can still use multiplier: {}", multiplier);  // Still accessible
    
    // FnMut: Mutable capture
    let mut count = 0;
    let mut increment = || {
        count += 1;  // Mutably borrows count
        count
    };
    println!("First call: {}", increment());
    println!("Second call: {}", increment());
    println!("Third call: {}", increment());
    println!("Final count: {}", count);
    
    // FnOnce: Move capture
    let vec = vec![1, 2, 3];
    let consume = move || {
        println!("Consumed vector: {:?}", vec);  // vec is moved into closure
        vec  // Returns ownership
    };
    let returned_vec = consume();
    // consume();  // ❌ Error: can't call twice (FnOnce)
    // println!("{:?}", vec);  // ❌ Error: vec was moved
    println!("Returned: {:?}", returned_vec);
    
    // Closures as function parameters
    fn apply_twice<F>(f: F, x: i32) -> i32
    where
        F: Fn(i32) -> i32,  // F must implement Fn trait
    {
        f(f(x))
    }
    
    let double = |x| x * 2;
    let result = apply_twice(double, 5);
    println!("\napply_twice(double, 5) = {}", result);  // (5 * 2) * 2 = 20
    
    // Returning closures (requires Box)
    fn make_adder(n: i32) -> Box<dyn Fn(i32) -> i32> {
        Box::new(move |x| x + n)
    }
    
    let add_5 = make_adder(5);
    let add_10 = make_adder(10);
    println!("\nadd_5(3) = {}", add_5(3));
    println!("add_10(3) = {}", add_10(3));
}

closures_demo();

---

## 🔬 Live Code Exploration

### Basic Iterator Operations

In [ ]:
// Basic iterator operations and patterns

fn basic_iterators_demo() {
    println!("=== Basic Iterator Operations ===");
    
    let numbers = vec![1, 2, 3, 4, 5, 6, 7, 8, 9, 10];
    
    // Basic iteration
    println!("Original numbers: {:?}", numbers);
    
    // Map: transform each element
    let squares: Vec<i32> = numbers
        .iter()
        .map(|x| x * x)
        .collect();
    println!("Squares: {:?}", squares);
    
    // Filter: keep elements that match condition
    let evens: Vec<&i32> = numbers
        .iter()
        .filter(|&x| x % 2 == 0)
        .collect();
    println!("Even numbers: {:?}", evens);
    
    // Chain operations
    let processed: Vec<i32> = numbers
        .iter()
        .filter(|&x| x % 2 == 0)  // Keep evens
        .map(|x| x * x)           // Square them
        .filter(|&x| x > 10)      // Keep if > 10
        .collect();
    println!("Even squares > 10: {:?}", processed);
    
    // Enumerate: add indices
    let indexed: Vec<(usize, &i32)> = numbers
        .iter()
        .enumerate()
        .collect();
    println!("With indices: {:?}", indexed);
    
    // Take and skip
    let first_three: Vec<&i32> = numbers
        .iter()
        .take(3)
        .collect();
    println!("First 3: {:?}", first_three);
    
    let skip_first_three: Vec<&i32> = numbers
        .iter()
        .skip(3)
        .collect();
    println!("Skip first 3: {:?}", skip_first_three);
    
    // Zip: combine two iterators
    let letters = vec!['a', 'b', 'c', 'd', 'e'];
    let zipped: Vec<(&i32, &char)> = numbers
        .iter()
        .zip(letters.iter())
        .collect();
    println!("Zipped: {:?}", zipped);
}

basic_iterators_demo();

In [ ]:
// Consumer methods and closure patterns

fn consumers_and_closures_demo() {
    println!("\n=== Consumer Methods and Closures ===");
    
    let numbers = vec![1, 2, 3, 4, 5, 6, 7, 8, 9, 10];
    
    // Fold: accumulate values
    let sum = numbers
        .iter()
        .fold(0, |acc, x| acc + x);
    println!("Sum using fold: {}", sum);
    
    // Reduce: similar to fold but uses first element as initial value
    let product = numbers
        .iter()
        .cloned()
        .reduce(|acc, x| acc * x)
        .unwrap_or(1);
    println!("Product using reduce: {}", product);
    
    // Find: get first matching element
    let first_even = numbers
        .iter()
        .find(|&x| x % 2 == 0);
    println!("First even number: {:?}", first_even);
    
    // Position: get index of first matching element
    let position = numbers
        .iter()
        .position(|&x| x > 5);
    println!("Position of first number > 5: {:?}", position);
    
    // Any and all
    let has_even = numbers.iter().any(|&x| x % 2 == 0);
    let all_positive = numbers.iter().all(|&x| x > 0);
    println!("Has even numbers: {}", has_even);
    println!("All positive: {}", all_positive);
    
    // Count
    let even_count = numbers
        .iter()
        .filter(|&x| x % 2 == 0)
        .count();
    println!("Count of even numbers: {}", even_count);
    
    // For each (side effects)
    println!("\nPrinting each number with for_each:");
    numbers
        .iter()
        .filter(|&x| x % 3 == 0)
        .for_each(|x| println!("  Multiple of 3: {}", x));
    
    // Closure examples with different capture modes
    let multiplier = 3;
    
    // Closure that captures by reference
    let multiplied: Vec<i32> = numbers
        .iter()
        .map(|x| x * multiplier)  // Captures multiplier by reference
        .collect();
    println!("\nMultiplied by {}: {:?}", multiplier, multiplied);
    
    // Closure that moves captured values
    let prefix = "Number: ".to_string();
    let formatted: Vec<String> = numbers
        .iter()
        .take(3)
        .map(move |x| format!("{}{}", prefix, x))  // Moves prefix into closure
        .collect();
    println!("Formatted (first 3): {:?}", formatted);
    // Note: prefix is no longer available here due to move
    
    // Complex closure example
    let words = vec!["hello", "world", "rust", "programming", "is", "fun"];
    let long_words: Vec<String> = words
        .iter()
        .filter(|word| word.len() > 4)  // Keep long words
        .map(|word| word.to_uppercase()) // Convert to uppercase
        .collect();
    println!("\nLong words (uppercase): {:?}", long_words);
}

consumers_and_closures_demo();

In [ ]:
// Creating custom iterators

// Custom iterator for Fibonacci sequence
struct Fibonacci {
    current: u64,
    next: u64,
}

impl Fibonacci {
    fn new() -> Self {
        Fibonacci {
            current: 0,
            next: 1,
        }
    }
}

impl Iterator for Fibonacci {
    type Item = u64;
    
    fn next(&mut self) -> Option<Self::Item> {
        let current = self.current;
        
        self.current = self.next;
        self.next = current + self.next;
        
        // Prevent overflow by stopping at large numbers
        if current > 1_000_000 {
            None
        } else {
            Some(current)
        }
    }
}

// Custom iterator for a range with step
struct StepRange {
    current: i32,
    end: i32,
    step: i32,
}

impl StepRange {
    fn new(start: i32, end: i32, step: i32) -> Self {
        StepRange {
            current: start,
            end,
            step,
        }
    }
}

impl Iterator for StepRange {
    type Item = i32;
    
    fn next(&mut self) -> Option<Self::Item> {
        if (self.step > 0 && self.current >= self.end) || 
           (self.step < 0 && self.current <= self.end) {
            None
        } else {
            let current = self.current;
            self.current += self.step;
            Some(current)
        }
    }
}

// Custom collection with iterator
#[derive(Debug)]
struct NumberList {
    numbers: Vec<i32>,
}

impl NumberList {
    fn new() -> Self {
        NumberList {
            numbers: Vec::new(),
        }
    }
    
    fn add(&mut self, number: i32) {
        self.numbers.push(number);
    }
    
    // Return an iterator over even numbers only
    fn evens(&self) -> impl Iterator<Item = &i32> {
        self.numbers.iter().filter(|&x| x % 2 == 0)
    }
    
    // Return an iterator over numbers in reverse
    fn reversed(&self) -> impl Iterator<Item = &i32> {
        self.numbers.iter().rev()
    }
    
    // Return an iterator that yields pairs of consecutive numbers
    fn pairs(&self) -> impl Iterator<Item = (&i32, &i32)> {
        self.numbers.iter().zip(self.numbers.iter().skip(1))
    }
}

// Implement IntoIterator for NumberList
impl IntoIterator for NumberList {
    type Item = i32;
    type IntoIter = std::vec::IntoIter<i32>;
    
    fn into_iter(self) -> Self::IntoIter {
        self.numbers.into_iter()
    }
}

// Implement IntoIterator for reference to NumberList
impl<'a> IntoIterator for &'a NumberList {
    type Item = &'a i32;
    type IntoIter = std::slice::Iter<'a, i32>;
    
    fn into_iter(self) -> Self::IntoIter {
        self.numbers.iter()
    }
}

fn custom_iterators_demo() {
    println!("\n=== Custom Iterators ===");
    
    // Fibonacci iterator
    println!("First 10 Fibonacci numbers:");
    let fib_numbers: Vec<u64> = Fibonacci::new()
        .take(10)
        .collect();
    println!("{:?}", fib_numbers);
    
    // Step range iterator
    println!("\nNumbers from 0 to 20 with step 3:");
    let stepped: Vec<i32> = StepRange::new(0, 20, 3)
        .collect();
    println!("{:?}", stepped);
    
    println!("\nNumbers from 10 to 0 with step -2:");
    let stepped_down: Vec<i32> = StepRange::new(10, 0, -2)
        .collect();
    println!("{:?}", stepped_down);
    
    // Custom collection with iterators
    let mut list = NumberList::new();
    for i in 1..=10 {
        list.add(i);
    }
    
    println!("\nNumber list: {:?}", list);
    
    // Use custom iterator methods
    let evens: Vec<&i32> = list.evens().collect();
    println!("Even numbers: {:?}", evens);
    
    let reversed: Vec<&i32> = list.reversed().collect();
    println!("Reversed: {:?}", reversed);
    
    let pairs: Vec<(&i32, &i32)> = list.pairs().collect();
    println!("Consecutive pairs: {:?}", pairs);
    
    // Use IntoIterator implementation
    println!("\nIterating with for loop (by reference):");
    for number in &list {
        if *number % 3 == 0 {
            println!("  Multiple of 3: {}", number);
        }
    }
    
    // Consume the list with into_iter
    println!("\nConsuming list and doubling values:");
    let doubled: Vec<i32> = list
        .into_iter()
        .map(|x| x * 2)
        .collect();
    println!("Doubled: {:?}", doubled);
    // Note: list is no longer available here
}

custom_iterators_demo();

In [ ]:
// TODO: Complete the data processing pipeline using iterators

#[derive(Debug, Clone)]
struct Sale {
    id: u32,
    product: String,
    amount: f64,
    quantity: u32,
    region: String,
    month: u32,
}

impl Sale {
    fn new(id: u32, product: String, amount: f64, quantity: u32, region: String, month: u32) -> Self {
        Sale { id, product, amount, quantity, region, month }
    }
    
    fn total_value(&self) -> f64 {
        self.amount * self.quantity as f64
    }
}

struct SalesAnalyzer {
    sales: Vec<Sale>,
}

impl SalesAnalyzer {
    fn new(sales: Vec<Sale>) -> Self {
        SalesAnalyzer { sales }
    }
    
    // Find top N products by total sales value
    fn top_products(&self, n: usize) -> Vec<(String, f64)> {
        use std::collections::HashMap;
        
        let mut product_totals: HashMap<String, f64> = HashMap::new();
        
        // Group by product and sum total values
        for sale in &self.sales {
            *product_totals.entry(sale.product.clone()).or_insert(0.0) += sale.total_value();
        }
        
        // Convert to vector, sort by value, and take top N
        let mut products: Vec<(String, f64)> = product_totals.into_iter().collect();
        products.sort_by(|a, b| b.1.partial_cmp(&a.1).unwrap());
        products.into_iter().take(n).collect()
    }
    
    // Calculate total sales by region
    fn sales_by_region(&self) -> Vec<(String, f64)> {
        use std::collections::HashMap;
        
        self.sales
            .iter()
            .fold(HashMap::new(), |mut acc, sale| {
                *acc.entry(sale.region.clone()).or_insert(0.0) += sale.total_value();
                acc
            })
            .into_iter()
            .collect()
    }
    
    // Find sales above a certain threshold
    fn high_value_sales(&self, threshold: f64) -> Vec<&Sale> {
        self.sales
            .iter()
            .filter(|sale| sale.total_value() > threshold)
            .collect()
    }
    
    // Calculate monthly trends
    fn monthly_totals(&self) -> Vec<(u32, f64)> {
        use std::collections::HashMap;
        
        let mut monthly_totals: HashMap<u32, f64> = HashMap::new();
        
        self.sales
            .iter()
            .for_each(|sale| {
                *monthly_totals.entry(sale.month).or_insert(0.0) += sale.total_value();
            });
        
        let mut result: Vec<(u32, f64)> = monthly_totals.into_iter().collect();
        result.sort_by_key(|&(month, _)| month);
        result
    }
    
    // Complex analysis: Average sale value by product category (first letter)
    fn avg_sale_by_category(&self) -> Vec<(char, f64)> {
        use std::collections::HashMap;
        
        let mut category_data: HashMap<char, (f64, u32)> = HashMap::new();
        
        self.sales
            .iter()
            .filter_map(|sale| sale.product.chars().next().map(|c| (c, sale.total_value())))
            .for_each(|(category, value)| {
                let entry = category_data.entry(category).or_insert((0.0, 0));
                entry.0 += value;
                entry.1 += 1;
            });
        
        category_data
            .into_iter()
            .map(|(category, (total, count))| (category, total / count as f64))
            .collect()
    }
    
    // Generate summary statistics
    fn summary_stats(&self) -> (f64, f64, f64, usize) {
        let values: Vec<f64> = self.sales
            .iter()
            .map(|sale| sale.total_value())
            .collect();
        
        let total: f64 = values.iter().sum();
        let count = values.len();
        let average = if count > 0 { total / count as f64 } else { 0.0 };
        
        let max = values.iter().cloned().fold(f64::NEG_INFINITY, f64::max);
        
        (total, average, max, count)
    }
    
    // Chain multiple operations for complex analysis
    fn complex_analysis(&self) -> Vec<String> {
        self.sales
            .iter()
            .filter(|sale| sale.total_value() > 1000.0)  // High-value sales
            .filter(|sale| sale.month >= 6)              // Second half of year
            .map(|sale| {
                format!(
                    "{} in {} (Month {}): ${:.2}",
                    sale.product,
                    sale.region,
                    sale.month,
                    sale.total_value()
                )
            })
            .collect()
    }
}

fn data_processing_demo() {
    println!("\n=== Data Processing Pipeline Demo ===");
    
    // Create sample sales data
    let sales = vec![
        Sale::new(1, "Laptop".to_string(), 999.99, 2, "North".to_string(), 3),
        Sale::new(2, "Mouse".to_string(), 29.99, 10, "South".to_string(), 3),
        Sale::new(3, "Keyboard".to_string(), 79.99, 5, "East".to_string(), 4),
        Sale::new(4, "Monitor".to_string(), 299.99, 3, "West".to_string(), 4),
        Sale::new(5, "Laptop".to_string(), 1199.99, 1, "North".to_string(), 5),
        Sale::new(6, "Tablet".to_string(), 399.99, 4, "South".to_string(), 6),
        Sale::new(7, "Phone".to_string(), 699.99, 6, "East".to_string(), 7),
        Sale::new(8, "Headphones".to_string(), 149.99, 8, "West".to_string(), 8),
        Sale::new(9, "Laptop".to_string(), 899.99, 3, "North".to_string(), 9),
        Sale::new(10, "Mouse".to_string(), 39.99, 15, "South".to_string(), 10),
    ];
    
    let analyzer = SalesAnalyzer::new(sales);
    
    // Top products analysis
    println!("\n📊 Top 3 Products by Total Sales:");
    let top_products = analyzer.top_products(3);
    for (i, (product, total)) in top_products.iter().enumerate() {
        println!("  {}. {} - ${:.2}", i + 1, product, total);
    }
    
    // Sales by region
    println!("\n🌍 Sales by Region:");
    let mut region_sales = analyzer.sales_by_region();
    region_sales.sort_by(|a, b| b.1.partial_cmp(&a.1).unwrap());
    for (region, total) in region_sales {
        println!("  {}: ${:.2}", region, total);
    }
    
    // High-value sales
    println!("\n💰 High-Value Sales (>$1000):");
    let high_value = analyzer.high_value_sales(1000.0);
    for sale in high_value {
        println!("  {} x{} {} in {} - ${:.2}", 
                sale.product, sale.quantity, sale.product, sale.region, sale.total_value());
    }
    
    // Monthly trends
    println!("\n📈 Monthly Sales Totals:");
    let monthly = analyzer.monthly_totals();
    for (month, total) in monthly {
        println!("  Month {}: ${:.2}", month, total);
    }
    
    // Average by category
    println!("\n📋 Average Sale Value by Product Category (First Letter):");
    let mut category_avg = analyzer.avg_sale_by_category();
    category_avg.sort_by_key(|&(category, _)| category);
    for (category, avg) in category_avg {
        println!("  Category '{}': ${:.2}", category, avg);
    }
    
    // Summary statistics
    let (total, average, max, count) = analyzer.summary_stats();
    println!("\n📊 Summary Statistics:");
    println!("  Total Sales: ${:.2}", total);
    println!("  Average Sale: ${:.2}", average);
    println!("  Highest Sale: ${:.2}", max);
    println!("  Number of Sales: {}", count);
    
    // Complex analysis
    println!("\n🔍 Complex Analysis (High-value sales in H2):");
    let complex_results = analyzer.complex_analysis();
    for result in complex_results {
        println!("  {}", result);
    }
}

data_processing_demo();